In [1]:
import pandas as pd
import numpy as np
from github_search import python_tokens
from mlutil.feature_extraction import embeddings
from mlutil import prototype_selection
import mlutil
from mlutil.feature_extraction import embeddings 
import transformers

In [2]:
%cd ..

/home/kuba/Projects/github_search


In [3]:
python_files_df = pd.read_csv('data/python_files.csv')

In [4]:
import pycg

In [5]:
import ast
import astunparse

In [6]:
example_file = python_files_df['content'].iloc[1]
print(example_file)

#!/usr/bin/env python3
# Author: Rico Sennrich

"""Use operations learned with learn_bpe.py to encode a new text.
The text will not be smaller, but use only a fixed vocabulary, with rare words
encoded as variable-length sequences of subword units.

Reference:
Rico Sennrich, Barry Haddow and Alexandra Birch (2016). Neural Machine Translation of Rare Words with Subword Units.
Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (ACL 2016). Berlin, Germany.
"""

import sys
import codecs
import argparse
import functools
from collections import defaultdict

class BPE(object):
    def encode(self, orig):
        """Encode word based on list of BPE merge operations, which are applied consecutively
        """
        word = tuple(orig) + ('</w>',)
        pairs = get_pairs(word)

        while True:
            bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float('inf')))
            if bigram not in self.bpe_codes:
                break
     

In [7]:
astunparse.unparse(ast.parse(example_file).body[-2])

"\n\ndef get_pairs(word):\n    'Return set of symbol pairs in a word.\\n\\n    word is represented as tuple of symbols (symbols being variable-length strings)\\n    '\n    pairs = set()\n    prev_char = word[0]\n    for char in word[1:]:\n        pairs.add((prev_char, char))\n        prev_char = char\n    return pairs\n"

In [8]:
fn_code = ast.parse(example_file).body[-2]

In [9]:
astunparse.unparse(fn_code.body[3])

'\nfor char in word[1:]:\n    pairs.add((prev_char, char))\n    prev_char = char\n'

In [10]:
astunparse.unparse(fn_code)

"\n\ndef get_pairs(word):\n    'Return set of symbol pairs in a word.\\n\\n    word is represented as tuple of symbols (symbols being variable-length strings)\\n    '\n    pairs = set()\n    prev_char = word[0]\n    for char in word[1:]:\n        pairs.add((prev_char, char))\n        prev_char = char\n    return pairs\n"

In [11]:
astunparse.unparse(fn_code)

"\n\ndef get_pairs(word):\n    'Return set of symbol pairs in a word.\\n\\n    word is represented as tuple of symbols (symbols being variable-length strings)\\n    '\n    pairs = set()\n    prev_char = word[0]\n    for char in word[1:]:\n        pairs.add((prev_char, char))\n        prev_char = char\n    return pairs\n"

In [12]:
fn_code.name

'get_pairs'

In [13]:
astunparse.unparse(fn_code.args)

'word\n'

In [14]:
%load_ext autoreload

In [15]:
import pyan.modvis

In [16]:
visitor = pyan.modvis.ImportVisitor(contents=[example_file])

In [17]:
visitor.prepare_graph()

In [18]:
graph_options = {
        "draw_defines": False,  # we have no defines edges
        "draw_uses": True,
        "colored": False,
        "grouped_alt": False,
        "grouped": True,
        "nested_groups": True,
        "annotated": False,
    }

In [19]:
import logging
logger = logging.getLogger()

In [20]:
graph = pyan.visgraph.VisualGraph.from_visitor(visitor, options=graph_options, logger=logger)

In [21]:
example_file

'#!/usr/bin/env python3\n# Author: Rico Sennrich\n\n"""Use operations learned with learn_bpe.py to encode a new text.\nThe text will not be smaller, but use only a fixed vocabulary, with rare words\nencoded as variable-length sequences of subword units.\n\nReference:\nRico Sennrich, Barry Haddow and Alexandra Birch (2016). Neural Machine Translation of Rare Words with Subword Units.\nProceedings of the 54th Annual Meeting of the Association for Computational Linguistics (ACL 2016). Berlin, Germany.\n"""\n\nimport sys\nimport codecs\nimport argparse\nimport functools\nfrom collections import defaultdict\n\nclass BPE(object):\n    def encode(self, orig):\n        """Encode word based on list of BPE merge operations, which are applied consecutively\n        """\n        word = tuple(orig) + (\'</w>\',)\n        pairs = get_pairs(word)\n\n        while True:\n            bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float(\'inf\')))\n            if bigram not in self.bpe_co

In [22]:
# %load notebooks/construct_call_graph.py
#!/usr/bin/env python

'''
generates call graph of given python code file
in dot format input for graphviz.

limitations:
* statically tried to figure out functions calls
* does not understand classes
* algorithm is naive and may not statically find
  all cases
'''

import sys
import parser
import symbol, token
import pprint
import optparse

try: s = set()
except: import sets; set = sets.Set

def annotate_ast_list(ast_list):
    code = ast_list[0]
    if code in symbol.sym_name: code = symbol.sym_name[code]
    else: code = token.tok_name[code]
    ast_list[0] = code

    for index, item in enumerate(ast_list):
        if index == 0: continue
        if isinstance(item, list):
            ast_list[index] = annotate_ast_list(item) 
    return ast_list
 
def get_atom_name(atom):
    first_child = atom[1]
    first_child_code = first_child[0]
    if first_child_code != token.NAME: return None
    return first_child[1]

def get_fn_call_data(ast_list):
    if len(ast_list) < 3: return None
    first_child, second_child = ast_list[1:3]
    first_child_code = first_child[0]
    if first_child_code != symbol.atom: return None
    fn_name = get_atom_name(first_child)

    second_child_code = second_child[0]
    if second_child_code != symbol.trailer: return None
    
    if len(second_child) < 3: return None
    if second_child[1][0] == token.LPAR and second_child[-1][0] == token.RPAR:
        return fn_name
    else: return None

def find_fn_call(ast_list, calls):
    code = ast_list[0]
    if code == symbol.power:
        fn_name = get_fn_call_data(ast_list)
        if fn_name != None and getattr(__builtins__, fn_name, None) == None: calls.add(fn_name) 
   
    for item in ast_list[1:]:
        if isinstance(item, list):
            find_fn_call(item, calls)

def process_fn(fn_ast_list, call_graph):
    dummy, dummy, func_name = fn_ast_list[:3]
    dummy, func_name = func_name

    calls = set()
    find_fn_call(fn_ast_list, calls)

    call_graph[func_name] = list(calls)

def construct_call_graph(ast_list, call_graph):
    code = ast_list[0]
    if code == symbol.funcdef:
        process_fn(ast_list, call_graph)

    for item in ast_list[1:]:
        if isinstance(item, list):
            construct_call_graph(item, call_graph)

    return call_graph

def generate_dot_code(python_code):
    ast_list = ast.parse(python_code).body
    #annotated_ast_list = annotate_ast_list(ast_list)
    #pprint.pprint(annotated_ast_list)

    call_graph = {}
    construct_call_graph(ast_list, call_graph)
    #pprint.pprint(call_graph)

    dot = []

    dot.append("digraph G {")
    dot.append("rankdir=LR")
    for from_fn, to_fns in call_graph.items():
        if not to_fns:
            dot.append('%s;' % from_fn)

        for to_fn in to_fns:
            if to_fn not in call_graph: continue
            dot.append('%s -> %s;' % (from_fn, to_fn))
    dot.append("}")

    return '\n'.join(dot)

dot_code = generate_dot_code(example_file)


In [23]:
code = """
def foo():
    bar()
"""

In [24]:
function_ast = (ast.parse(code).body[0])

In [25]:

ast.parse(code).body[0].name

'foo'

In [26]:
exprs = ast.parse(code).body[0].body

In [27]:
expr = exprs[0]

In [29]:
def get_assign_calls(expr):
    return

In [30]:
def get_calls(function_ast):
    return frozenset(
        expr.value.func.id if type(expr.value.func) is ast.Name else expr.value.func.attr
        for expr in function_ast.body
        if type(expr) is ast.Expr or type(expr) is ast.Assign and type(expr.value) is ast.Call
    )

In [31]:
expr_ast = ast.parse("word = tuple(orig) + ''").body[0]

In [32]:
expr_ast.value

In [402]:
def get_calls_from_expr_or_assign(expr):
    try:
        if type(expr) is ast.Name:
            return [expr.id]
        if type(expr) is ast.Call:
            return get_calls_from_expr_or_assign(expr.func) + get_calls_from_expr_or_assign(expr.args)
        elif type(expr) is ast.Attribute:
            return [astunparse.unparse(expr).strip()]
        elif type(expr) is ast.BinOp:
            return get_calls_from_expr_or_assign(expr.left) + get_calls_from_expr_or_assign(expr.right)
        elif type(expr) is ast.Expr:
            return get_calls_from_expr_or_assign(expr.value)
        elif type(expr) is ast.Assign:
            return get_calls_from_expr_or_assign(expr.value)
        elif type(expr) is ast.While or type(expr) is ast.If:
            return (
                get_calls_from_expr_or_assign(expr.test) +
                get_calls_from_expr_or_assign(expr.orelse) +
                get_calls_from_expr_or_assign(expr.body)
            )
        elif type(expr) is ast.For:
            return (
                get_calls_from_expr_or_assign(expr.target) +
                get_calls_from_expr_or_assign(expr.body) +
                get_calls_from_expr_or_assign(expr.iter)
            )
        elif type(expr) is ast.Try:
            return get_calls_from_expr_or_assign(expr.body)
        elif type(expr) is list:
            return [res for subexpr in expr for res in get_calls_from_expr_or_assign(subexpr)]
        else:
            return []
    except Exception as e:
        print(str(astunparse.unparse(expr)), ' ', str(e))

In [403]:
expr_ast.value.right

In [404]:
expr_ast.value

In [405]:
astunparse.unparse(expr_ast.value)

"(tuple(orig) + '')\n"

In [406]:
expr_ast.value.left.func.id

'tuple'

In [407]:
get_calls_from_expr_or_assign(expr_ast)

['tuple', 'orig']

In [408]:
expr_ast.value.left

In [409]:
encode_code = """def encode(self, orig):
    word = tuple(orig) + ('</w>',)
    pairs = get_pairs(word)

    while True:
        bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float('inf')))
        if bigram not in self.bpe_codes:
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
                new_word.append(first + second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # don't print end-of-word symbols
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>', ''),)

    return word
"""

In [410]:
encode_ast = ast.parse(encode_code)

In [411]:
assign_body = encode_ast.body[0].body[0]

In [412]:
assign_body.value

In [413]:
astunparse.unparse(assign_body.value)

"(tuple(orig) + ('</w>',))\n"

In [414]:
get_calls(function_ast)

frozenset({'bar'})

In [415]:
class_def = ast.parse(code).body[0]

In [416]:
class_def.body

In [417]:
def zip_dicts(dicts):
    res = {}
    for d in dicts:
        res = {**res, **d}
    return res

In [418]:
zip_dicts([{"1": "2"}, {"2": "3"}])

{'1': '2', '2': '3'}

In [419]:
def get_ast_function_calls(code_ast, calls={}):
    if type(code_ast) is ast.FunctionDef:
        return {code_ast.name: sum([get_calls_from_expr_or_assign(expr) for expr in code_ast.body], [])}
    else:
        return zip_dicts([get_ast_function_calls(item) for item in code_ast.body])

In [420]:
sum([[],[]], [])

[]

In [421]:
def get_function_calls(code):
    code_ast = ast.parse(code)
    return get_ast_function_calls(code_ast)

In [422]:
e = ast.parse("tuple(orig)").body[0]

In [423]:
e.value.func

In [424]:
e = ast.parse("functools.lru_cache(maxsize=65536)(self.encode)").body[0]

In [425]:
get_calls_from_expr_or_assign(e)

['functools.lru_cache', 'self.encode']

In [426]:
ev  = e.value.func.func

In [427]:
ev.value.id

'functools'

In [428]:
e = ast.parse('functools.lru_cache').body[0]

In [429]:
e.value

In [430]:
astunparse.unparse(e)

'\nfunctools.lru_cache\n'

In [431]:
print(astunparse.unparse(class_def))



class BPE(object):

    def encode(self, orig):
        word = (tuple(orig) + ('</w>',))
        pairs = get_pairs(word)
        while True:
            bigram = min(pairs, key=(lambda pair: self.bpe_codes.get(pair, float('inf'))))
            if (bigram not in self.bpe_codes):
                break
            (first, second) = bigram
            new_word = []
            i = 0
            while (i < len(word)):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break
                if ((word[i] == first) and (i < (len(word) - 1)) and (word[(i + 1)] == second)):
                    new_word.append((first + second))
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
       

In [432]:
get_function_calls(class_def)

{'encode': ['tuple',
  'orig',
  'get_pairs',
  'word',
  'min',
  'pairs',
  'bigram',
  'word.index',
  'first',
  'i',
  'new_word.extend',
  'j',
  'new_word.append',
  'new_word.append',
  'first',
  'second',
  'tuple',
  'new_word',
  'new_word',
  'get_pairs',
  'word',
  'word[(- 1)].endswith'],
 '__init__': ['functools.lru_cache', 'self.encode', 'dict', 'separator'],
 'segment': ['word',
  'self.encode',
  'word',
  'item',
  'output.append',
  'item',
  'self.separator',
  'output.append',
  'sentence.split']}

In [433]:

e = ast.parse(
"new_word.extend(word[i:j])"
).body[0]

In [360]:
get_calls_from_expr_or_assign(e)

['new_word.extend']

In [361]:
encode_ast.body[0].body

In [362]:
get_ast_function_calls(encode_ast.body[0])

{'encode': ['tuple', 'get_pairs']}

In [363]:
get_ast_function_calls(ast.parse(encode_code).body[0])

{'encode': ['tuple', 'get_pairs']}

In [364]:
get_function_calls(code)

{'encode': ['tuple', 'get_pairs'],
 '__init__': ['functools.lru_cache', 'dict', 'separator'],
 'segment': ['word', 'sentence.split']}

In [365]:
assert get_function_calls(code) == {"foo": frozenset(["bar"])}

AssertionError: 

In [366]:
items = ast.parse(code).body

In [367]:
code = """
class BPE(object):
    def encode(self, orig):
        word = tuple(orig) + ('</w>',)
        pairs = get_pairs(word)

        while True:
            bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float('inf')))
            if bigram not in self.bpe_codes:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
                    new_word.append(first + second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)

        # don't print end-of-word symbols
        if word[-1] == '</w>':
            word = word[:-1]
        elif word[-1].endswith('</w>'):
            word = word[:-1] + (word[-1].replace('</w>', ''),)

        return word

    def __init__(self, codes, separator='@@'):
        self.encode = functools.lru_cache(maxsize=65536)(self.encode)
        self.bpe_codes = [tuple(item.split()) for item in codes]
        # some hacking to deal with duplicates (only consider first instance)
        self.bpe_codes = dict([(code,i) for (i,code) in reversed(list(enumerate(self.bpe_codes)))])

        self.separator = separator

    def segment(self, sentence):
        output = []
        for word in sentence.split():
            new_word = self.encode(word)

            for item in new_word[:-1]:
                output.append(item + self.separator)
            output.append(new_word[-1])

        return ' '.join(output)
"""

In [368]:
print(example_file)

#!/usr/bin/env python3
# Author: Rico Sennrich

"""Use operations learned with learn_bpe.py to encode a new text.
The text will not be smaller, but use only a fixed vocabulary, with rare words
encoded as variable-length sequences of subword units.

Reference:
Rico Sennrich, Barry Haddow and Alexandra Birch (2016). Neural Machine Translation of Rare Words with Subword Units.
Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (ACL 2016). Berlin, Germany.
"""

import sys
import codecs
import argparse
import functools
from collections import defaultdict

class BPE(object):
    def encode(self, orig):
        """Encode word based on list of BPE merge operations, which are applied consecutively
        """
        word = tuple(orig) + ('</w>',)
        pairs = get_pairs(word)

        while True:
            bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float('inf')))
            if bigram not in self.bpe_codes:
                break
     

In [434]:
get_function_calls(code)

{'encode': ['tuple',
  'orig',
  'get_pairs',
  'word',
  'min',
  'pairs',
  'bigram',
  'word.index',
  'first',
  'i',
  'new_word.extend',
  'j',
  'new_word.append',
  'new_word.append',
  'first',
  'second',
  'tuple',
  'new_word',
  'new_word',
  'get_pairs',
  'word',
  'word[(- 1)].endswith'],
 '__init__': ['functools.lru_cache', 'self.encode', 'dict', 'separator'],
 'segment': ['word',
  'self.encode',
  'word',
  'item',
  'output.append',
  'item',
  'self.separator',
  'output.append',
  'sentence.split']}

In [443]:
items = ast.parse("a.append(1)").body
expr = items[0].value

In [447]:
get_calls_from_expr_or_assign(expr)

['a.append']

In [446]:
expr.func

In [442]:
astunparse.unparse(expr)

'a.append(1)\n'

In [128]:
type(expr)

_ast.Expr

In [129]:
expr.attr

AttributeError: 'Expr' object has no attribute 'attr'

In [130]:
astunparse.unparse(expr)

'\nbar()\n'

In [131]:
items[0].body[0].value

AttributeError: 'FunctionDef' object has no attribute 'value'

In [132]:
call.func.id

NameError: name 'call' is not defined